# Assignment 3 (Maksim Kaledin)

## Problem 1

Let $G(V,E)$ be a complete directed weighted graph with tiles as vertices $V$, and euclidean distances between tiles $i,j$ as weights $c_{ij}$. Denote by $\delta^+(v)$ all input edges and by $\delta^-(v)$ all output edges. Consider $f(e)\geq 0$ to be a flow through the edge $e$, $f(i,j)$ (in shorter notation, $f_{ij}$) corresponds to amount of ground transferred from tile $i$ to $j$. We also need to define demands and supplies $b_i=g'_i-g_i$ for each tile $i$, where $g_i$ is the current amount of ground at $i$ and $g'_i$ is the goal one. Negative $b_i$ note that there it too much ground while the positive ones show the demand for additional ground. Summing up, we have the following $LP$ problem.

$$\text{min}_{f_{ij}} \sum_{i\neq j}f_{ij},~ s.t.$$

$$\sum_{e \in \delta^+(i)}f(e) - \sum_{e \in \delta^-(i)}f(e) = b_i \quad \forall i\in V.$$

The constraints could be expressed easily since $G$ is a complete graph.

In [102]:
import cvxpy as cv
import numpy as np
import scipy as sp

n=20 # number of tiles

#tile positions
tiles=np.zeros((2,n*(n-1)))
for i in np.arange(4):
    for j in np.arange(5):
        tiles[0][i*5+j]=j
        tiles[1][i*5+j]=i

costs=np.array([sp.linalg.norm(tiles[:,j]-tiles[:,i],2) for j in np.arange(n) for i in np.arange(n) if not(j==i) ])


#demands
b=np.array([1,1,-4,-4,-4,
            1,1,-4,-14,-4,
            6,1,1,-4,1,
            6,6,6,1,6])

#constraints
A=np.zeros((n,n*(n-1)))

               
for i in np.arange(n):
    A[i,i*(n-1):(i+1)*(n-1)]=-1 # input edges

    if(i>0):
        A[i,np.array([k*(n-1)+i-1 for k in np.arange(i)])]=1 #for k<i
    
    if(i+1<n):
        A[i,np.array([k*(n-1)+i for k in np.arange(i+1,n)])]=1 #for k>i

print(A[:,:15])#just check if it is correctly generated

[[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

In [143]:
#CVX model
#variables
f = cv.Variable(n*(n-1))
#sequentially f_i1,...,f_ij,...,f_in with i != any of j

flowConstraints = [A*f == b, f>=0]
flowObj = cv.Minimize(costs*f)

flowProblem = cv.Problem(flowObj,flowConstraints)
flowProblem.solve(solver="GUROBI")

print ("status:",flowProblem.status)
print ("optimal value", flowProblem.value)
#print ("optimal var", f.value.reshape((n,n-1)))
res=f.value.reshape((n,n-1))


for i in np.arange(n):
    
    for el in [j for j in np.arange(n-1) if res[i,j]>0]:
        if(i<el):
            print("move from "+"("+str(i % 5)+","+str(i // 5)+")"+" to "+"("+str((el-1) % 5)+","+str((el-1) // 5)+")"+": "+str(res[i,el]))
        else:
            print("move from "+"("+str(i % 5)+","+str(i // 5)+")"+" to "+"("+str(el % 5)+","+str(el // 5)+")"+": "+str(res[i,el]))


status: optimal
optimal value 95.49106383667822
move from (1,0) to (0,0): 1.0
move from (2,0) to (1,0): 2.0
move from (2,0) to (3,0): 1.0
move from (2,0) to (4,0): 1.0
move from (3,0) to (3,1): 4.0
move from (4,0) to (1,3): 1.0
move from (4,0) to (2,3): 3.0
move from (2,1) to (3,1): 2.0
move from (2,1) to (4,1): 1.0
move from (2,1) to (3,2): 1.0
move from (3,1) to (0,2): 7.0
move from (3,1) to (3,2): 5.0
move from (3,1) to (0,3): 2.0
move from (4,1) to (2,2): 4.0
move from (2,2) to (4,2): 6.0
move from (3,2) to (0,3): 4.0
move from (4,2) to (2,3): 3.0


Note that Gurobi returned an integer solution. However there are fractional, according to CVX, even if we consider 1e-10 equals zero due to float point errors.

In [145]:
#CVX model
#variables
f = cv.Variable(n*(n-1))
#sequentially f_i1,...,f_ij,...,f_in with i != any of j

flowConstraints = [A*f == b, f>=0]
flowObj = cv.Minimize(costs*f)

flowProblem = cv.Problem(flowObj,flowConstraints)
flowProblem.solve()

print ("status:",flowProblem.status)
print ("optimal value", flowProblem.value)
#print ("optimal var", f.value.reshape((n,n-1)))
res=f.value.reshape((n,n-1))


for i in np.arange(n):
    
    for el in [j for j in np.arange(n-1) if res[i,j]>0]:
        if(i<el):
            print("move from "+"("+str(i % 5)+","+str(i // 5)+")"+" to "+"("+str((el-1) % 5)+","+str((el-1) // 5)+")"+": "+str(res[i,el]))
        else:
            print("move from "+"("+str(i % 5)+","+str(i // 5)+")"+" to "+"("+str(el % 5)+","+str(el // 5)+")"+": "+str(res[i,el]))


status: optimal
optimal value 95.4910638297555
move from (0,0) to (0,0): 2.08110600224e-11
move from (0,0) to (3,0): 1.81793079316e-10
move from (0,0) to (4,0): 1.95782968581e-11
move from (0,0) to (3,1): 9.0965702718e-11
move from (0,0) to (4,1): 6.00977727133e-12
move from (0,0) to (3,2): 4.45700990122e-11
move from (1,0) to (0,0): 0.532171543474
move from (1,0) to (1,0): 1.72445090092e-11
move from (1,0) to (3,0): 9.19387883737e-10
move from (1,0) to (4,0): 2.57596393401e-10
move from (1,0) to (0,1): 2.3491179658e-11
move from (1,0) to (3,1): 3.60445804567e-10
move from (1,0) to (4,1): 1.08317101262e-10
move from (1,0) to (0,2): 1.44558748348e-11
move from (1,0) to (3,2): 1.72158797332e-10
move from (1,0) to (4,2): 6.16516944325e-11
move from (1,0) to (0,3): 1.05373463301e-11
move from (2,0) to (0,0): 0.467828454981
move from (2,0) to (1,0): 1.53217154407
move from (2,0) to (2,0): 3.02184683988e-11
move from (2,0) to (3,0): 0.999999995756
move from (2,0) to (4,0): 0.999999991822
mov

## Problem 3

Let $x_{ij}$ be the binary variable which equals $1$, if student $i$ is assigned to room together with student $j$ for $i,j=1,..,20$ (considering $i<j$). Denote by $p_{ij}$ the (symmetric) preference matrix. The corresponding ILP problem can be formulated as

$$
\max_{x_{ij}} \sum_{i < j}p_{ij}x_{ij},~ s.t.
$$
$$
\sum_{i=2,..,20}\sum_{j < i} x_{ij} \leq 10,
$$
$$
\sum_{j<i}x_{ij} + \sum_{j>i}x_{ji} = 1 \quad \forall i=1,..,20.
$$



In [187]:
import gurobipy as grb
n=20#students
m=10#rooms

#preferences
prefs_matrix=np.random.uniform(0,1,(n,n))
prefs=dict([((i,j),prefs_matrix[i,j]) for i in np.arange(1,n) for j in np.arange(i) ])

#model definition
model=grb.Model("Assignment")
indexes=[(i,j) for i in np.arange(1,n) for j in np.arange(i)]

#variables
xij= model.addVars(indexes,vtype=grb.GRB.BINARY)

model.setObjective(xij.prod(prefs),grb.GRB.MAXIMIZE)

model.addConstr(xij.sum(),grb.GRB.LESS_EQUAL, m, "rooms")

for i in np.arange(1,n):
    expr = grb.LinExpr(np.array(np.ones(i)), [xij[i,j] for j in np.arange(i)])
    expr.addTerms(np.array(np.ones(n-1-i)), [xij[j,i] for j in np.arange(i+1,n)])
    model.addConstr(expr,grb.GRB.EQUAL,1)

model.optimize()
#np.array([xij[i,j].x for i in np.arange(1,n) for j in np.arange(i)])
for el in indexes:
    el_lst=list(el)
    if(not(xij[el_lst[0],el_lst[1]].x==0)):
        print(el,xij[el_lst[0],el_lst[1]].x)

Optimize a model with 20 rows, 190 columns and 551 nonzeros
Variable types: 0 continuous, 190 integer (190 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve time: 0.00s
Presolved: 20 rows, 190 columns, 532 nonzeros
Variable types: 0 continuous, 190 integer (190 binary)

Root relaxation: objective 9.073658e+00, 20 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       9.0736583    9.07366  0.00%     -    0s

Explored 0 nodes (20 simplex iterations) in 0.01 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 9.07366 
Pool objective bound 9.07366

Optimal solution found (tolerance 1.00e-04)
Best objective 9.073658302856e+00, best bound 9.073658302856e+00, gap 0.0000%
(7, 2) 1.0
(8, 3) 1.0

In [180]:
xij[1,0]

<gurobi.Var C0>